In [88]:
from typing import Literal
from pprint import pprint
import time

import numpy as np
import sklearn

import seaborn as sns
from tqdm import tqdm

In [4]:
type Sample = list[int | float]
type Data = list[Sample]

type Target = int | float
type Targets = list[Target]

In [5]:
# Activation functions

from abc import ABC, abstractmethod


class ActivationBase(ABC):
    @abstractmethod
    def calc(self, x: Sample) -> list[float]:
        """Apply the activation function to an layer output"""
        pass

#######################################


class ReLU(ActivationBase):
    def calc(self, x) -> list[float]:
        return np.maximum(0, x)


class ELU(ActivationBase):
    def __init__(self, alpha: float = 1.0) -> None:
        self.alpha = alpha
    
    def calc(self, x: Sample) -> list[float]:
        return np.where(x > 0, x, self.alpha * (np.exp(x) - 1))


class Sigmoid(ActivationBase):
    def calc(self, x: Sample) -> list[float]:
        return 1 / (1 + np.exp(-x))


class Tahn(ActivationBase):
    def calc(self, x: Sample) -> list[float]:
        return np.tanh(x)


class Softmax(ActivationBase):
    """returns model 'probabilities' for each class"""

    def calc(self, x: Sample) -> list[float]:
        
        # optimization: make numbers in an array from -inf to 0 because of a np.exp growing
        # and returns an array of floats from 0.0 to 1.0
        max_value = np.max(x)
        x -= max_value

        exp_values = np.exp(x)
        return exp_values / np.sum(exp_values)

In [6]:
# Loss functions


class LossBase(ABC):
    @abstractmethod
    def calc(self, x: Sample, y: Targets) -> float:
        """Apply the loss function to an output layer"""
        pass


class MSELoss(LossBase):
    """For regression"""
    def __init__(self, reduction: Literal['sum', 'mean'] = 'mean') -> None:
        self.reduction = reduction
    
    def calc(self, x: Sample, y: Targets) -> float:

        x = np.array(x)
        loss = (x - y) ** 2

        if self.reduction == 'sum':
            return np.sum(loss)
        elif self.reduction == 'mean':
            return np.mean(loss)


class CrossEntropy(LossBase):
    """For classification"""
    def calc(self, x: Sample, y: Targets) -> float:
        return -np.sum(y * np.log(x))


In [7]:
a = [1, 100, 22, 99]

f = Softmax()

b = f.calc(a)
print(b)
print(sum(b))

[7.39262147e-44 7.31058579e-01 9.74950551e-35 2.68941421e-01]
1.0


In [8]:
class Dataset:
    def __init__(self, data: Data, target_list: Targets) -> None:
        self.data: Data = data
        self._len = len(data)
        self.target_list: Targets = target_list

    def __len__(self) -> int:
        return self._len
    
    def __getitem__(self, index) -> Sample:
        return self.data[index]
    
    def __iter__(self):
        return iter(self.data)


In [94]:
class Linear:
    def __init__(self, n_inputs: int, n_neurons: int, activation: ActivationBase) -> None:
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        
        self.weights = self._init_weights()
        self.biases = self._init_biases()
        self.output = []

        self.activation = activation
    
    def _init_weights(self) -> list[float]:
        return np.random.randn(self.n_inputs, self.n_neurons) * 0.1
    
    def _init_biases(self):
        return np.random.randn(self.n_neurons)
    
    def forward(self, inputs) -> None:
        # print(f"{inputs = }")
        # print(f"{self.weights = }")
        # print(f"{self.biases = }")
        output = np.dot(inputs, self.weights)

        # print(f"{output = }")

        output += self.biases
        # output = output + self.biases


        self.output = self.activation.calc(output)
        # print(f"{self.output = }")


type Layers = list[Linear]

In [95]:
class Model:
    def __init__(self, layers: Layers, loss: LossBase, n_epoch: int = 1, verbose: bool = True):
        self.layers = layers
        self._layers_len = len(layers)
        self.loss = loss
        
        self.n_epoch = n_epoch
        self.verbose = verbose

    def fit(self, dataset: Dataset):
        losses = []

        range_epoch = range(self.n_epoch)
        if self.verbose:
            range_epoch = tqdm(range_epoch, desc="epochs", position=0)

        for epoch in range_epoch:
            for i,sample in enumerate(dataset):
                # print("Layer", 1, '\n\n')

                self.layers[0].forward(inputs=sample)
                
                for j in range(1, self._layers_len):
                    # print("Layer", j+1, '\n\n')
                    
                    self.layers[j].forward(inputs=self.layers[j-1].output)

                targets = dataset.target_list[i]
                
                # Calc loss
                loss = self.calc_loss(targets=targets)
                losses.append(loss)
    
    def predict(self):
        ...
    
    def calc_loss(self, targets: Target) -> float:
        output_layer = self.layers[-1]
        output = output_layer.output

        loss = self.loss.calc(x=output, y=targets)
        return loss

    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights.append(layer.weights)
        return weights

In [96]:
X_train = [
    # [1,2,3,4],
    [4,3,2,1]
]
y_train = [1,]


X_val = [
    [1,2,3,4],
    # [4,3,2,1]
]
y_val = [1,]


train_dataset = Dataset(data=X_train, target_list=y_train)
# val_dataset = Dataset(data=val_data)

In [104]:
layers = [
    Linear(4,2, activation=ReLU()),
    Linear(2,4, activation=ReLU()),
    Linear(4,1, activation=Softmax()),
]

model = Model(layers=layers, loss=MSELoss(), n_epoch=10, verbose=1)

In [105]:
model.fit(dataset=train_dataset)

epochs: 100%|██████████| 100000/100000 [00:03<00:00, 27419.57it/s]


In [ ]:
model.layers[-1].

In [ ]:
model.weights